In [35]:
import torch
import sys
sys.path.append('/Users/daniyalkhan/Documents/AI/DL/genAI-builders/llm_from_scratch')
from utils import SelfAttention_v2
from torch import nn

In [5]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
x_2 = inputs[1] #A
d_in = inputs.shape[1] #B
d_out = 2 #C

In [6]:
sa_v2 = SelfAttention_v2(d_in, d_out)

In [7]:
queries = sa_v2.W_query(inputs) #A
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[0.1686, 0.1683, 0.1681, 0.1649, 0.1636, 0.1665],
        [0.1716, 0.1565, 0.1570, 0.1722, 0.1768, 0.1657],
        [0.1718, 0.1569, 0.1573, 0.1719, 0.1763, 0.1657],
        [0.1689, 0.1591, 0.1595, 0.1713, 0.1751, 0.1661],
        [0.1739, 0.1655, 0.1654, 0.1651, 0.1642, 0.1659],
        [0.1672, 0.1556, 0.1563, 0.1743, 0.1805, 0.1661]],
       grad_fn=<SoftmaxBackward0>)


In [9]:
# We can now use PyTorch's tril function to create a mask where the values above the diagonal are zero:

In [10]:
context_length = inputs.shape[0]
torch.ones(context_length, context_length)

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [11]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [12]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1686, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1716, 0.1565, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1718, 0.1569, 0.1573, 0.0000, 0.0000, 0.0000],
        [0.1689, 0.1591, 0.1595, 0.1713, 0.0000, 0.0000],
        [0.1739, 0.1655, 0.1654, 0.1651, 0.1642, 0.0000],
        [0.1672, 0.1556, 0.1563, 0.1743, 0.1805, 0.1661]],
       grad_fn=<MulBackward0>)


In [13]:
row_sums = masked_simple.sum(dim=1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5230, 0.4770, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3536, 0.3227, 0.3237, 0.0000, 0.0000, 0.0000],
        [0.2564, 0.2415, 0.2421, 0.2600, 0.0000, 0.0000],
        [0.2084, 0.1984, 0.1983, 0.1979, 0.1969, 0.0000],
        [0.1672, 0.1556, 0.1563, 0.1743, 0.1805, 0.1661]],
       grad_fn=<DivBackward0>)


In [14]:
# The result is an attention weight matrix where the attention weights above the diagonal are zeroed out and where the rows sum to 1.
# While we could be technically done with implementing causal attention at this point, we can take advantage of a mathematical property of the softmax function.
# We can implement the computation of the masked attention weights more efficiently in fewer steps.
# The softmax function converts its inputs into a probability distribution.
# When negative infinity values (-∞) are present in a row, the softmax function treats them as zero probability.
# (Mathematically, this is because e -∞ approaches 0.)
# We can implement this more efficient masking "trick" by creating a mask with 1's above the diagonal and then replacing these 1's with negative infinity (-inf) values:

In [15]:
print(attn_scores)

tensor([[ 0.0628,  0.0598,  0.0586,  0.0310,  0.0204,  0.0450],
        [-0.1866, -0.3169, -0.3125, -0.1821, -0.1444, -0.2363],
        [-0.1763, -0.3053, -0.3010, -0.1758, -0.1400, -0.2275],
        [-0.1569, -0.2420, -0.2384, -0.1373, -0.1067, -0.1806],
        [ 0.0603, -0.0093, -0.0100, -0.0133, -0.0205, -0.0060],
        [-0.2613, -0.3628, -0.3570, -0.2027, -0.1535, -0.2712]],
       grad_fn=<MmBackward0>)


In [16]:
torch.triu(torch.ones(context_length, context_length))

tensor([[1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.]])

In [17]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
print(mask)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])


In [18]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[ 0.0628,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.1866, -0.3169,    -inf,    -inf,    -inf,    -inf],
        [-0.1763, -0.3053, -0.3010,    -inf,    -inf,    -inf],
        [-0.1569, -0.2420, -0.2384, -0.1373,    -inf,    -inf],
        [ 0.0603, -0.0093, -0.0100, -0.0133, -0.0205,    -inf],
        [-0.2613, -0.3628, -0.3570, -0.2027, -0.1535, -0.2712]],
       grad_fn=<MaskedFillBackward0>)


In [19]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5230, 0.4770, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3536, 0.3227, 0.3237, 0.0000, 0.0000, 0.0000],
        [0.2564, 0.2415, 0.2421, 0.2600, 0.0000, 0.0000],
        [0.2084, 0.1984, 0.1983, 0.1979, 0.1969, 0.0000],
        [0.1672, 0.1556, 0.1563, 0.1743, 0.1805, 0.1661]],
       grad_fn=<SoftmaxBackward0>)


In [20]:
# As we can see based on the output, the values in each row sum to 1, and no further normalization is necessary.
# Masking in Transformers sets scores for future tokens to a large negative value, making their influence in the softmax calculation effectively zero.
# The softmax function then recalculates attention weights only among the unmasked tokens.
# This process ensures no information leakage from masked tokens, focusing the model solely on the intended data.
# We could now use the modified attention weights to compute the context vectors via context_vec = attn_weights @ values.
# However, in the next section, we first cover another minor tweak to the causal attention mechanism that is useful for reducing overfitting when training LLMs.

In [21]:
# MASKING ADDITIONAL ATTENTION WEIGHTS WITH DROPOUT

In [22]:
example = torch.ones(6, 6) #B
print(example)

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])


In [23]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) #A
example = torch.ones(6, 6) #B
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [24]:
# When applying dropout to an attention weight matrix with a rate of 50%, half of the elements in the matrix are randomly set to zero.
# To compensate for the reduction in active elements, the values of the remaining elements in the matrix are scaled up by a factor of 1/0.5 =2.
# This scaling is crucial to maintain the overall balance of the attention weights, ensuring that the average influence of the attention mechanism remains consistent during both the training and inference phases.

In [25]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7071, 0.6455, 0.6474, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4829, 0.4842, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3969, 0.0000, 0.3958, 0.0000, 0.0000],
        [0.0000, 0.3113, 0.3126, 0.3486, 0.3609, 0.0000]],
       grad_fn=<MulBackward0>)


In [26]:
# As we can see above, the resulting attention weight matrix now has additional elements zeroed out and the remaining ones rescaled.

In [27]:
### IMPLEMENTING A COMPACT CAUSAL ATTENTION CLASS

In [29]:
# Before we begin, one more thing is to ensure that the code can handle batches consisting of more than one input.
# This will ensure that the CausalAttention class supports the batch outputs produced by the data loader we implemented earlier.

In [32]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) 
# This results in a 3D tensor consisting of 2 input texts with 6 tokens each, where each token is a 3-dimensional embedding vector.

torch.Size([2, 6, 3])


In [33]:
# Step 1: Compared to the previous SelfAttention_v1 class, we added a dropout layer.
# Step 2: The register_buffer call is also a new addition (more information is provided in the following text).
# Step 3: We transpose dimensions 1 and 2, keeping the batch dimension at the first position (0).
# Step 4: In PyTorch, operations with a trailing underscore are performed in-place, avoiding unnecessary memory copies

In [36]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

In [38]:
# The use of register_buffer in PyTorch is not strictly necessary for all use cases but offers several advantages here.
# For instance, when we use the CausalAttention class in our LLM, buffers are automatically moved to the appropriate device (CPU or GPU) along with our model, which will be relevant when training the LLM in future chapters.
# This means we don't need to manually ensure these tensors are on the same device as your model parameters, avoiding device mismatch errors.

In [39]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([2, 6, 2])


In [41]:
print(context_vecs)
# As we can see, the resulting context vector is a 3D tensor where each token is now represented by a 2D embedding

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
